In [17]:
###################importing libraries##############################
from __future__ import print_function
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from openpyxl import load_workbook


In [2]:
####################Reading the csv Files####################################
x_train=pd.read_csv(r'G:\Class_project\New_folder\A_train_samples.csv')
y_train=pd.read_csv(r'G:\Class_project\New_folder\A_train_labels.csv')
x_test=pd.read_csv(r'G:\Class_project\New_folder\A_test_samples.csv')
y_test=pd.read_csv(r'G:\Class_project\New_folder\A_test_labels.csv')
x_train=x_train[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29', 'F30', 'F31',
       'F32', 'F33', 'F34', 'F35', 'F36', 'F37', 'F38', 'F39', 'F40', 'F41',
       'F42']]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
type(y_train)

(175341, 42)
(175341, 25)
(175341, 42)
(175341, 1)


pandas.core.frame.DataFrame

In [3]:
####################scaling the training Data######################################
scaler = MinMaxScaler()
x_train_scl = scaler.fit_transform(x_train)
x_test_scl = scaler.transform(x_test)
#x_train_scl=to_dataframe(x_train_scl)
x_train = pd.DataFrame(data=x_train_scl)
x_test=pd.DataFrame(data=x_test_scl)

In [4]:
################Creating Function to fit and evaluate the data###################
def fit_and_evaluate(t_x, val_x, t_y, val_y, EPOCHS=100, BATCH_SIZE=1000):
    results = model.fit(t_x, t_y, epochs=EPOCHS, batch_size=BATCH_SIZE, 
              verbose=1, validation_split=0.1)  
    print("Val Score: ", model.evaluate(val_x, val_y))
    return results

In [5]:
y_train=y_train[['B','C5','C10']]


In [16]:
#############Creating loop for each column in y_train#################
t=y_train.columns
count=0
for i in range(y_train.shape[1]):
    y_train_1=y_train[t[i]]
    y_train_1=pd.DataFrame(y_train_1)
    model = Sequential()
    # in the first layer, you must specify the expected input data shape:
    # here, 43-dimensional vectors.
    model.add(Dense(64, activation='relu', input_dim=42))

    model.add(Dense(128, activation='relu'))

    model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.1)))
    model.add(BatchNormalization())
    #model.add(Dropout(0.1))
    model.add(Dense(11, activation='softmax'))

    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    n_folds=1
    #save the model history in a list after fitting so that we can plot later
    model_history = [] 

    for i in range(n_folds):
        print("Training on Fold: ",i+1)
        t_x, val_x, t_y, val_y = train_test_split(x_train, y_train_1, test_size=0.1,
                                                  random_state = np.random.randint(1,1000, 1)[0])
        model_history.append(fit_and_evaluate(t_x, val_x, t_y, val_y, 2,1000))
        score = model.evaluate(x_test,y_test,batch_size=1000)
        y_pred=model.predict_classes(x_test)
        if count==0:
            writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
            pd.DataFrame(y_pred).to_excel(writer, sheet_name='Sheet_name_1')
            writer.save()
        else:
            writer = pd.ExcelWriter('output.xlsx', engine='openpyxl')
            writer.book = load_workbook('output.xlsx')
            writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
            reader = pd.read_excel(r'output.xlsx')
            pd.DataFrame(y_pred).to_excel(writer,startcol=count+1)
            writer.close()
        count=+1
        

Training on Fold:  1
Train on 142025 samples, validate on 15781 samples
Epoch 1/2
142025/142025 [==============================] - 1s 7us/step - loss: 5.1100 - accuracy: 0.6896 - val_loss: 2.0007 - val_accuracy: 0.5119
Epoch 2/2
17535/17535 [==============================] - 0s 17us/step
Val Score:  [1.3122733804756057, 0.5382948517799377]
175341/175341 [==============================] - 0s 2us/step
Training on Fold:  1
Train on 142025 samples, validate on 15781 samples
Epoch 1/2
142025/142025 [==============================] - 1s 9us/step - loss: 5.3117 - accuracy: 0.6583 - val_loss: 2.0772 - val_accuracy: 0.4903
Epoch 2/2
17535/17535 [==============================] - 0s 17us/step
Val Score:  [1.4955496320161583, 0.47835758328437805]
175341/175341 [==============================] - 0s 2us/step
Training on Fold:  1
Train on 142025 samples, validate on 15781 samples
Epoch 1/2
142025/142025 [==============================] - 1s 8us/step - loss: 5.4068 - accuracy: 0.6201 - val_loss: 2.18